# **Classification** des **séries temporelles** avec `sktime`

## Métadonnées

- **Expérience nº :** 1.3.4
- **Date :** 15/11/2023
- **Heure :** 23:26
- **Données :** `ready_Evaporation.csv`
- **Tri :** index chronologique
- **Découpage :** `TimeSeriesSplit`
- **Mise à l'échelle :** non
- **Rééquilibrage :** non
- **Conversion :** `to_sktime_dataset`
- **Algorithme :** ROCKET

## Sommaire

1. Initialisation
2. Découpage
3. Conversion
4. Modélisation
5. Évaluation

## 1. Initialisation

In [1]:
# Importation des bibliothèques et modules nécessaires au fonctionnement de ce notebook

import pandas as pd
import numpy as np

from sklearn.metrics import classification_report

In [2]:
# Importation du jeu de données et enregistrement dans le DataFrame `df`

df = pd.read_csv("../../../data/9df/Omar/ready_Evaporation.csv", index_col = 1).sort_index()
df = df.drop(columns = "Unnamed: 0")

In [3]:
# Inspection de la structure de `df`

df.head()

,MinTemp,MaxTemp,Rainfall,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,...,year,month,day,LocationNum,WindGustDirNum,WindDir9amNum,WindDir3pmNum,Latitude,Longitude,CodeRegionNum
Date,,,,,,,,,,,,,,,,,,,,,
2009-01-01,18.800000,38.4,0.0,13.1,50.000000,22.0,13.0,46.0,20.0,1010.4,...,2009,1,1,38,0.000000,5.890486,5.497787,-31.667778,116.015,5
2009-01-02,22.000000,39.1,0.0,13.2,52.000000,4.0,31.0,38.0,23.0,1008.3,...,2009,1,2,38,0.000000,2.356194,3.926991,-31.667778,116.015,5
2009-01-03,15.900000,35.6,0.0,12.7,43.473477,13.0,28.0,58.0,29.0,1012.1,...,2009,1,3,38,2.896869,9.424778,4.319690,-31.667778,116.015,5
2009-01-04,12.456393,37.0,0.0,13.3,43.000000,24.0,20.0,45.0,19.0,1016.6,...,2009,1,4,38,5.497787,5.497787,5.890486,-31.667778,116.015,5
2009-01-05,21.800000,38.6,0.0,13.2,59.000000,15.0,15.0,35.0,15.0,1014.1,...,2009,1,5,38,0.000000,5.890486,5.105088,-31.667778,116.015,5


In [4]:
# Inspection de la structure de `df`

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2646 entries, 2009-01-01 to 2017-06-25
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   MinTemp         2646 non-null   float64
 1   MaxTemp         2646 non-null   float64
 2   Rainfall        2646 non-null   float64
 3   Sunshine        2646 non-null   float64
 4   WindGustSpeed   2646 non-null   float64
 5   WindSpeed9am    2646 non-null   float64
 6   WindSpeed3pm    2646 non-null   float64
 7   Humidity9am     2646 non-null   float64
 8   Humidity3pm     2646 non-null   float64
 9   Pressure9am     2646 non-null   float64
 10  Pressure3pm     2646 non-null   float64
 11  Cloud9am        2646 non-null   float64
 12  Cloud3pm        2646 non-null   float64
 13  Temp9am         2646 non-null   float64
 14  Temp3pm         2646 non-null   float64
 15  RainToday       2646 non-null   int64  
 16  RainTomorrow    2646 non-null   int64  
 17  year            2646 no

## 2. Découpage

In [5]:
# Découpage de `df` sur l'axe des colonnes : séparation des variables explicatives (`data`) et cible (`target`)

data = df.drop(columns = "RainTomorrow")
target = df["RainTomorrow"]

In [6]:
# Découpage de `data` et de `target` sur l'axe des lignes : séparation des jeux d'entraînement (`*_train`) et de test (`*_test`) avec le splitter `TimeSeriesSplit`

from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits = 5) ## Nous fixons le paramètre `n_splits` à 5 afin d'avoir une répartition de 80 / 20 entre les jeux d'entraînement et de test, respectivement.

for train_index, test_index in tss.split(data):
    X_train, X_test = data.iloc[train_index, :], data.iloc[test_index,:]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]

## 3. Conversion

In [7]:
# Conversion des jeux `X_train` et `X_test` au format `sktime` avec la fonction `to_sktime_dataset` de la bibliothèque `tslearn`

from tslearn.utils import to_sktime_dataset

X_train = to_sktime_dataset(X_train)
X_test = to_sktime_dataset(X_test)

`tslearn` est un autre _package_ Python qui fournit des outils d'apprentissage automatique pour l'analyse de séries temporelles (réf. 1).

Il propose, dans son module `utils`, la fonction `to_sktime_dataset`, qui permet de convertir un jeu de données vers un format de scitype `"Panel"` et de mtype `"nested_univ"`.

Ce format fait partie de ceux admis par `sktime` (réf. 2, § 1.2, § 3.2.2) et correspond notamment à celui du jeu de données dans sa démonstration de ROCKET (réf. 3).

Références :
1. https://tslearn.readthedocs.io/en/stable/index.html
2. https://www.sktime.net/en/stable/examples/AA_datatypes_and_datasets.html
3. https://www.sktime.net/en/latest/examples/transformation/rocket.html

In [8]:
X_train.head()

,dim_0
0,0 18.800000 1 38.400000 2 0...
1,0 22.000000 1 39.100000 2 0...
2,0 15.900000 1 35.600000 2 0...
3,0 12.456393 1 37.000000 2 0...
4,0 21.800000 1 38.600000 2 0...


Cette conversion nécessite le changement de paradigme suivant :
- Chaque date (ligne) est considérée comme une INSTANCE (un échantillon).
- Pour chaque instance, toutes les VARIABLES EXPLICATIVES sont concaténées en une seule (`dim_0`).
- Chaque concaténation est considérée comme une série de POINTS TEMPORELS, disposés _horizontalement_.

Cette formulation n'est pas intuitive : en effet, il est plus naturel de définir une série temporelle _verticalement_, en suivant l'index chronologique.

Elle est néanmoins utile car elle permet d'adapter le jeu de données à la méthode ROCKET.

Elle s'inspire de l'analyse des images en tant que séries temporelles par le déroulement de leur contour sur un hypothétique axe temporel, permettant par la suite d'y appliquer des méthodes réservées aux séries temporelles :

![image.png](https://www.sktime.net/en/v0.20.0/_images/from-shapes-to-time-series.png)

## 4. Modélisation

In [8]:
# Instanciation d'un transformateur ROCKET

from sktime.transformations.panel.rocket import Rocket

rocket = Rocket()

In [9]:
# Récupération des paramètres initiaux du transformateur ROCKET

rocket.get_params()

{'n_jobs': 1, 'normalise': True, 'num_kernels': 10000, 'random_state': None}

In [10]:
# Ajustement du transformateur ROCKET au jeu d'entraînement

rocket.fit(X_train)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Rocket()

In [11]:
# Récupération des paramètres ajustés du transformateur ROCKET

rocket.get_fitted_params()

{}

In [12]:
# Application du transformateur ROCKET au jeu d'entraînement

X_train_transform = rocket.transform(X_train)

In [13]:
# Instanciation d'un modèle classificateur linéaire

from sklearn.linear_model import RidgeClassifierCV

clf_rocket_ridge = RidgeClassifierCV(alphas = np.logspace(-3, 3, 10)) ## Pour ce paramétrage initial, nous suivons l'exemple fourni par `sktime`, qui fixe 10 alphas, de -3 à 3, à équidistance sur l'échelle logarithmique.

In [14]:
# Récupération des paramètres initiaux du modèle

clf_rocket_ridge.get_params()

{'alphas': array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
        4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
        2.15443469e+02, 1.00000000e+03]),
 'class_weight': None,
 'cv': None,
 'fit_intercept': True,
 'scoring': None,
 'store_cv_values': False}

In [15]:
# Entraînement du modèle

clf_rocket_ridge.fit(X_train_transform, y_train)

RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]))

In [16]:
# Récupération des paramètres ajustés du modèle

clf_rocket_ridge.get_params()

{'alphas': array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
        4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
        2.15443469e+02, 1.00000000e+03]),
 'class_weight': None,
 'cv': None,
 'fit_intercept': True,
 'scoring': None,
 'store_cv_values': False}

In [17]:
# Application du transformateur ROCKET au jeu de test

X_test_transform = rocket.transform(X_test)

In [18]:
# Réalisation des prédictions

y_pred = clf_rocket_ridge.predict(X_test_transform)

## 5. Évaluation

In [19]:
# Élaboration de la matrice de confusion

pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames = ["Classe prédite"])

Classe prédite,0,1
Classe réelle,,
0,326,20
1,30,65


In [20]:
# Élaboration du rapport de classification

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.94      0.93       346
           1       0.76      0.68      0.72        95

    accuracy                           0.89       441
   macro avg       0.84      0.81      0.83       441
weighted avg       0.88      0.89      0.88       441

